# 视频url爬取
- 此处分为两种类型，视频的播放源来自两种，第一种为mooc1，第二种为mooc2
- 这里我们想要做的是将其分为两种形式来进行爬取

In [1]:
import time
import pandas as pd
# from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import asyncio
import aiohttp
import os # 这个库是用来进行文件处理操作的
import moviepy.editor as mp # 提取出视频中的音频部分
from seleniumwire import webdriver
from tqdm import tqdm

In [19]:
# 我们将会pick两个demo进行测试
# 前两个的播放源为mooc2，最后一个播放源为mooc1
url_c1 = "https://www.icourse163.org/course/GXU-1205891801"
c_name1 = "木与人类文明"
url_c2 = "https://www.icourse163.org/course/HEBYKDX-1454685171"
c_name2 = "卫生应急管理"
url_c3 = "https://www.icourse163.org/course/SCU-20012"
c_name3 = "化妆品赏析与应用"

In [20]:
# 全部数据源
df_200 = pd.read_excel('200个选课_V2.xlsx')
df_200

,课程名称,评价人数,是否可报名,课程类别,详细类别,开课单位,开课老师,课程url,评分,可爬取
0,木与人类文明,2846,1,理学工学农学,农林园艺,广西大学,高伟,https://www.icourse163.org/course/GXU-1205891801,4.9,1.0
1,动物学,199,1,理学工学农学,农林园艺,华中农业大学,姬伟,https://www.icourse163.org/course/HZAU-1002478006,4.7,1.0
2,茶叶感官审评,494,1,理学工学农学,农林园艺,安徽农业大学,戴前颖,https://www.icourse163.org/course/AHAU-1206699836,4.8,1.0
3,禽生产学,120,1,理学工学农学,农林园艺,西北农林科技大学,王哲鹏,https://www.icourse163.org/course/NWSUAF-12066...,4.7,1.0
4,卫生应急管理,3805,1,医学与保健,医药卫生,哈尔滨医科大学,吴群红,https://www.icourse163.org/course/HEBYKDX-1454...,4.9,1.0
...,...,...,...,...,...,...,...,...,...,...
195,程序设计入门——C语言,11643,1,计算机,计算机,浙江大学,翁恺,https://www.icourse163.org/course/ZJU-199001,4.9,1.0
196,数据结构,2384,1,计算机,计算机,浙江大学,陈越,https://www.icourse163.org/course/ZJU-93001,4.9,1.0
197,面向对象程序设计——Java语言,1477,1,计算机,计算机,浙江大学,翁恺,https://www.icourse163.org/course/ZJU-1001542001,4.9,1.0
198,C语言程序设计进阶,1463,1,计算机,计算机,浙江大学,翁恺,https://www.icourse163.org/course/ZJU-200001,4.9,1.0


In [21]:
# 首先先处理登陆的操作
# 准备对应的url地址
url = "https://www.icourse163.org/"

# 下面主要先进行处理登陆的问题
person = {
    # "account" : "13539651927"     ,
    # "password": "qazmlp123456@...",
    "account" : "13501408005"     ,
    "password": "qazwsx123520",
}

# 首先创建一个driver实例对象
driver = webdriver.Chrome()
driver.maximize_window()

# 避开 chrome 检测
driver.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {
        "source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""
    }
)

# 打开对应的网站
driver.get(url)

# 登录
driver.find_element(by=By.CSS_SELECTOR,value="#j-topnav > div > a").click()
# 其他登录方式
driver.find_element(by=By.CSS_SELECTOR,value="div.ux-login-set-scan-code_ft > span").click()
# 手机号登录
driver.find_element(by=By.CSS_SELECTOR,value="div.ux-tabs-underline > ul > li:nth-child(2)").click()
driver.switch_to.frame(driver.find_element(by=By.CSS_SELECTOR,value="div.ux-login-urs-phone-wrap.f-pr > div > iframe"))

# 输入账号
driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[2]/div[2]/input").clear()
driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[2]/div[2]/input").send_keys(person["account"])
# 输入密码
driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[4]/div[2]/input[2]").clear()
driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[4]/div[2]/input[2]").send_keys(person["password"])

time.sleep(1)
# 登录
driver.find_element(by=By.XPATH,value="//div[@class='m-container']/div[@class='u-tab f-cb']//following::div[@class='f-cb loginbox']").click()

time.sleep(2)
driver.implicitly_wait(5)

## 编写函数

In [22]:
def merge_and_remove(path_1,path_2,sub_urls):
    # 首先获取所有的文件的名字
    #lst = os.listdir(path)
    # 这里是适用于mac系统的所以下面是len(lst)-1 win 系统下面为len(lst)即可
    name_lst = [f'{i}.ts' for i in range(len(sub_urls))]
    #print(name_lst)
    for file in tqdm(name_lst, desc="正在转换视频格式："):
        if os.path.exists(path_1 + path_2 + '/' +file):
            with open(path_1 + path_2 + '/' + file, 'rb') as f1:
                with open(path_1  + f"{path_2}.mp4", 'ab') as f2:
                    f2.write(f1.read())
        else:
            print("失败")

    for item in name_lst:
        # 删除所有的ts片段
        os.remove(path_1 + path_2 +'/'+item)
    # 删除装ts视频片段的文件夹
    os.removedirs(path_1 + path_2)

In [23]:
# 下面开始进行爬取
# course200_urls = df_200['课程url'].tolist()
# course200_names = df_200['课程名称'].tolist()
# course200_schools = df_200['开课单位'].tolist()
# course200_teachers = df_200['开课老师'].tolist()
#
# new_names = [t1+t2+t3 for t1 in course200_names for t2 in course200_schools for t3 in course200_teachers]

# course_keyi_name = []
# course_keyi_url = []

# 设置一个临时的urls list和一个names list
course200_urls = [url_c1,url_c2,url_c3]
new_names = [c_name1,c_name2,c_name3]

for url_c,c_name in zip(course200_urls[-1:],new_names[-1:]):
    # 首先对于课程页面的url
    # 这里需要重新改一下就是获取对应的course名称和单位和老师加上对应页面的url
    # 首先在当前的路径下判断有没有该课程文件夹，现在文件夹会主要存放两种类型，第一种类型为csv文件，第二种类型为mp4类型
    # 这个是为了方便后面看看第几个课出问题了
    print(c_name)
    dir_name = f'{c_name}_url_or_video'
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    driver.execute_script("window.open('{}');".format(url_c))
    # 切换到新打开的课程页面
    driver.switch_to.window(driver.window_handles[-1])
    # 已参加，继续学习
    driver.find_element(by=By.CSS_SELECTOR,value="div.course-enroll-info_course-enroll_buttons > div > span").click()
    # 关掉弹窗
    driver.find_element(by=By.CSS_SELECTOR,value="div.ux-modal_hd > a").click()
    # 课件
    driver.find_element(by=By.XPATH,value="/html/body/div[4]/div[2]/div[4]/div[1]/div[1]/ul/li[3]").click()

    # 下面开始点击对应的视频出来
    # 保存好当前页面的 url，后面需要反复访问
    curr = driver.current_url
    print("curr",curr)

    # 先把所有展开的收起
    already_click = driver.find_elements(by=By.XPATH,value="//div[@class='sourceList']/div/div[contains(text(),'视频')]/ancestor::div[@class='m-learnChapterNormal f-pr']/div[@class='titleBox j-titleBox f-cb']")
    if already_click:
        for i in already_click:
            i.click()

    # 章，后面需要不断更新
    chapters = driver.find_elements(by=By.XPATH,value="//div[@class='m-learnChapterList f-pr']//div[@class='m-learnChapterNormal f-pr']//div[@class='titleBox j-titleBox f-cb']//h3")
    #print("chapters",[i.text for i in chapters])

    # 维护一个用来索引的数组
    chapters_idx = [i for i in range(len(chapters))]
    #print("chapter_idx",chapters_idx)

    # 空列表保存视频 url(对于视频源为mooc1)
    video_urls = []
    names = []
    for idx in chapters_idx:

        # 更新当前章的对象
        chapters = driver.find_elements(by=By.XPATH,value="//div[@class='m-learnChapterList f-pr']//div[@class='m-learnChapterNormal f-pr']//div[@class='titleBox j-titleBox f-cb']//h3")
        chapters[idx].click()

        #print("idx",idx)
        time.sleep(0.5)

        # 当前章下的视频节，后面需要不断更新
        video_section = driver.find_elements(by=By.XPATH,value="//div[@class='sourceList']/div/div[contains(text(),'视频')]/parent::div")
        video_section_idx = [i for i in range(len(video_section))]
        #print("video_section",[i.text for i in video_section])

        for video_idx in video_section_idx:
            #print("video_idx",video_idx)

            # 在新窗口打开
            driver.execute_script("window.open('{}');".format(curr))

            # 切换窗口
            handles = driver.window_handles
            driver.switch_to.window(handles[-1])

            # 先把所有展开的收起，再从头开始点击
            already_click = driver.find_elements(by=By.XPATH,value="//div[@class='sourceList']/div/div[contains(text(),'视频')]/ancestor::div[@class='m-learnChapterNormal f-pr']/div[@class='titleBox j-titleBox f-cb']")
            if already_click:
                for i in already_click:
                    i.click()

            time.sleep(1)
            # 更新当前章的对象
            chapters = driver.find_elements(by=By.XPATH,value="//div[@class='m-learnChapterList f-pr']//div[@class='titleBox j-titleBox f-cb']//h3")
            chapters[idx].click()
            time.sleep(1)


            # 更新当前视频节对象
            video_section = driver.find_elements(by=By.XPATH,value="//div[@class='sourceList']/div/div[contains(text(),'视频')]/parent::div")
            time.sleep(1)
            names.append("{}+{}".format(chapters[idx].text,video_section[video_idx].get_attribute("title")))
            video_section[video_idx].click()



            time.sleep(1)
            # 获取视频 url,如果失败则刷新一下再获取
            try:
                # 这里表示的是mooc1的形式
                video_urls.append(driver.find_element(by=By.CSS_SELECTOR,value="div.u-edu-h5player-mainvideo.f-pa.j-mainVideo > video > source").get_attribute("src"))
            except:
                # 这里表示的是视频的播放源是mooc2的形式，此处我们就直接把视频给爬取出来
                # 思路： 爬取每个视频的ts片段的url =》下载对应的ts片段 =》 拼接ts片段变成mp4文件
                video_fragment_ts = []
                for request in driver.requests:
                    #print(request)
                    if ".m3u8" in request.url:
                        video_fragment_ts.append(request.url)
                # 实际上我们上面就提取到了m3u8文件的request url
                m3u8_request_url = video_fragment_ts[-1]
                # print(m3u8_request_url)
                # print(video_fragment_ts)
                sharing_url = m3u8_request_url[:m3u8_request_url.rfind('/')]
                # 下面先下载m3u8文件，然后拼接出所有的ts的片段，并且给名字
                resp = requests.get(m3u8_request_url)
                sub_urls = [u for u in resp.text.split('\n') if u[-3:] == '.ts']
                # 需要为每一个视频建立一个文件夹
                if not os.path.exists(f'./{dir_name}/{names[-1]}'):
                    os.makedirs(f'./{dir_name}/{names[-1]}')

                for m in range(len(sub_urls)):
                    req_url = sharing_url+'/'+sub_urls[m]
                    # 发送请求并且存放到对应的文件夹中
                    tp_resp = requests.get(req_url)
                    with open(f'./{dir_name}/{names[-1]}/{m}.ts','wb') as f:
                        f.write(tp_resp.content)

                # 上面我们实现了将每一个视频对应的所有的ts视频片段下载到课程文件夹中的视频文件夹
                # 下面我们将会实现将所有的ts合并为一个mp4文件 并且进行删除ts片段
                path_1 = f'./{dir_name}'
                path_2 = f'/{names[-1]}'
                # 合并为mp4文件并且将ts文件进行删除
                merge_and_remove(path_1,path_2,sub_urls)


            # 关掉当前窗口
            driver.close()

            # 切回一开始的窗口
            driver.switch_to.window(handles[1])

        # 重要!!!把一开始打开的章收起
        chapters = driver.find_elements(by=By.XPATH,value="//div[@class='m-learnChapterList f-pr']//div[@class='titleBox j-titleBox f-cb']//h3")
        chapters[idx].click()
    # 加上一个判断 如果url的
    if len(video_urls)!=0:
        # 合成一个表格
        pd.DataFrame({
        "names":names,
        "video_urls":video_urls,
        }).to_csv(f'./{dir_name}/{c_name}_video_url.csv',index=False)

    # 关闭课程的页面
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    # course_keyi_url.append(url_c)
    # course_keyi_name.append(c_name)

化妆品赏析与应用
curr https://www.icourse163.org/learn/SCU-20012?tid=1467129514#/learn/content


In [18]:
print(new_names.index(c_name))

1


In [27]:
len(video_urls)

69

In [28]:
len(names)

69

In [29]:
names

['第1讲 概说——化妆品的前世今生+视频1.1\xa0化妆品的概念',
 '第1讲 概说——化妆品的前世今生+视频化妆品的发展',
 '第1讲 概说——化妆品的前世今生+视频化妆品的特性',
 '第1讲 概说——化妆品的前世今生+视频化妆品的分类',
 '第1讲 概说——化妆品的前世今生+视频化妆品的吸收',
 '第1讲 概说——化妆品的前世今生+视频化妆品的选择与使用',
 '第2讲 揭开面纱——化妆品原料+视频13.1\xa0化妆品原料概念和分类',
 '第2讲 揭开面纱——化妆品原料+视频13.2\xa0化妆品基质原料',
 '第2讲 揭开面纱——化妆品原料+视频13.3\xa0化妆品粉质原料和水性原料',
 '第2讲 揭开面纱——化妆品原料+视频13.4\xa0化妆品表面活性剂',
 '第2讲 揭开面纱——化妆品原料+视频13.5\xa0化妆品辅助原料',
 '第3讲 天然与雕琢——化妆品配方、生产工艺+视频14.1\xa0膏霜的配方原理与生产',
 '第3讲 天然与雕琢——化妆品配方、生产工艺+视频14.2\xa0精华素、啫喱的配制与特色',
 '第3讲 天然与雕琢——化妆品配方、生产工艺+视频14.3\xa0彩妆的配方原理与生产',
 '第3讲 天然与雕琢——化妆品配方、生产工艺+视频14.4\xa0香波、护发素的配制与特色',
 '第4讲 秀发卫士——毛发类化妆品+视频2.1\xa0毛发的结构和功能',
 '第4讲 秀发卫士——毛发类化妆品+视频2.2\xa0毛发类化妆品的成分及选择要点',
 '第4讲 秀发卫士——毛发类化妆品+视频2.3\xa0毛发类化妆品使用的常见疑问',
 '第4讲 秀发卫士——毛发类化妆品+视频2.4\xa0护发攻略',
 '第5讲 温柔呵护——舒缓类化妆品+视频敏感性皮肤的概述1',
 '第5讲 温柔呵护——舒缓类化妆品+视频敏感性皮肤的概述2',
 '第5讲 温柔呵护——舒缓类化妆品+视频舒缓类化妆品原料及作用机制',
 '第5讲 温柔呵护——舒缓类化妆品+视频3.3\xa0舒缓类化妆品的选择与使用',
 '第5讲 温柔呵护——舒缓类化妆品+视频舒缓类化妆品的功效评价',
 '第6讲 UV不在——防晒类化妆品+视频紫外辐射及其对人类皮肤的基本损害',
 '第6讲 UV不在——防晒类化妆品+视频防晒类化妆品的历史形成和发展

In [10]:
os.removedirs('./木与人类文明_url_or_video/1 绪论+视频视频-1.1本课程中的木和文明')

In [9]:
for i,j in zip([1],[2]):
    print(i)
    print(j)

1
2


## testing

In [96]:
dir_name = "testing"
names = ['hahaha']
# 实际上我们上面就提取到了m3u8文件的request url
m3u8_request_url = "https://mooc2vod.stu.126.net/nos/hls/2021/10/11/66/ad19833d-3531-4cd7-9150-81199ee139a5_7.m3u8?ak=7909bff134372bffca53cdc2c17adc27a4c38c6336120510aea1ae1790819de8ebd9b5d43f43bc6f732667516cf10fad3296f41833121932ed36da2b405f08bb3059f726dc7bb86b92adbc3d5b34b1326959e1fd05d5a4fc919af812946cbac8b8ae77e29788836745b7125f174b3914"
sharing_url = m3u8_request_url[:m3u8_request_url.rfind('/')]
# 下面先下载m3u8文件，然后拼接出所有的ts的片段，并且给名字
resp = requests.get(m3u8_request_url)
sub_urls = [u for u in resp.text.split('\n') if u[-3:] == '.ts']
# 需要为每一个视频建立一个文件夹
if not os.path.exists(f'./{dir_name}/{names[-1]}'):
    os.makedirs(f'./{dir_name}/{names[-1]}')

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
}

for m in range(len(sub_urls)):
    req_url = sharing_url+'/'+sub_urls[m]
    #print(req_url)
    # 发送请求并且存放到对应的文件夹中
    tp_resp = requests.get(req_url,headers=headers,stream=True,verify=False)
    with open(f'./{dir_name}/{names[-1]}/{m}.ts','wb') as f:
        f.write(tp_resp.content)


In [99]:
path = f'./{dir_name}/{names[-1]}'
def merge_and_remove(path):
    # 首先获取所有的文件的名字
    lst = os.listdir(path)
    # 这里是适用于mac系统的所以下面是len(lst)-1 win 系统下面为len(lst)即可
    name_lst = [f'{i}.ts' for i in range(len(lst)-1)]
    #print(name_lst)
    for file in tqdm(name_lst, desc="正在转换视频格式："):
        if os.path.exists(path + '/' +file):
            with open(path + '/' + file, 'rb') as f1:
                with open(path + '/' + "damn.mp4", 'ab') as f2:
                    f2.write(f1.read())
        else:
            print("失败")
    for item in name_lst:
        os.remove(path+'/'+item)


merge_and_remove(path)

正在转换视频格式：: 100%|██████████| 99/99 [00:00<00:00, 844.28it/s]
